_**Note:** The first code cell of this notebook comes from my JupyterLab Templates folder._  **Template**: JLab_reporting.ipynb  
See Tim Paine's [JupyterLab Template Extension](https://github.com/timkpaine/jupyterlab_templates) for setting up the extension.

# _Notebook from **Template**: JLab_reporting.ipynb_

In [1]:
import sys
from pathlib import Path

#print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
#print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = Path(this_path).parent.as_posix()
    else:
        newp = Path(this_path).as_posix()
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        #print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

#DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown #display, HTML
##display(HTML("<style>.container { width:100% !important; }</style>"))
#def new_section(title):
#    style = "text-align:center;background:#c2d3ef;padding:20px;color:#ffffff;font-size:3em;width:98%"
#    return HTML('<div style="{}">{}</div>'.format(style, title))


# Filtered dir() for method discovery:
def filter_module_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work together
              to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]


def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy down two levels only."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))


# Extended utils for reporting:  ..................................................
import IPython
#print(f'IPython ver: {IPython.__version__}')
# '7.11.0'

from IPython import get_ipython

def is_lab_notebook():
    import re
    import psutil

    return any(re.search('jupyter-lab-script', x)
               for x in psutil.Process().parent().cmdline())
#print(f'is_lab_notebook() = { is_lab_notebook()}\n')

def ipy_set_next_input(cell_contents, replace_cell):
    if replace_cell:
        shell = get_ipython()

        payload = dict(
            source='set_next_input',
            text=cell_contents,
            replace=True,
        )
        shell.payload_manager.write_payload(payload, single=False)
    else:
        return Markdown(cell_contents)
    

def insert_md_div(div_text_format, div_data=None, replace_cell=False):
    """
    Output Markdown of string with inserted data.
    div_text_format, e.g.: 
               "this is my data: {}" if div_data=x or [x]
               "that too: {}, {}" if div_data=[x,y] or (x,y)
     => assume # place holders == # data items.
    """
    if div_data is None or '{' not in div_text_format:
        # todo: flag error? (bc function not needed)
        return Markdown(div_text_format)

    mismatch_err = 'Format placeholders != data items'
    if isinstance(div_data, (tuple, list)):
        if div_text_format.count('{}') != len(div_data):
            raise IndexError(mismatch_err)
        div = div_text_format.format(*div_data)
        
    elif isinstance(div_data, dict):
        if '{}' not in div_text_format:
            # eg, assume keys given as indices: 'a is {a}, b is {b}'
            div = div_text_format.format_map(div_data)
        else:
            if div_text_format.count('{}') != len(div_data.values()):
                raise IndexError(mismatch_err)
            div = div_text_format.format(*div_data.values())

    else:
        if div_text_format.count('{}') != 1:
            raise IndexError(mismatch_err)
        div = div_text_format.format(div_data)

    return ipy_set_next_input(div, replace_cell)


def insert_alert_div(div_class, div_header=None, div_text=None, 
                     use_class_as_header=True,
                     replace_cell=False):
    """
    use_class_as_header: if div_header=None, div_class is used if 
                         use_class_as_header=True
    Behaviour with `replace_cell=True`:
    The cell is overwritten with the output string:
    ```
    [x]
    alert_div('warning', 'Tip: ', 'some tip here', replace_cell=True)
    The [x] cell becomes:
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    However, the cell mode is still in 'code' not 'markdown', so the desired output
    is obtained only after *manually* changing the cell mode to Markdown.
    
    If `replace_cell=False (default)`, the Markdown string is displayed as any code output cell,
    i.e. below the code cell.
    """
    accepted = ['info', 'warning', 'danger']
    if div_class.lower() not in accepted:
        s = f'<div class="alert"><b>Wrong class:</b> `div_class` not in {accepted}</div>'
        return Markdown(s)
    
    div_class = div_class.lower()
    div_class = 'alert-' + div_class
    div = f'<div class="alert {div_class}"><p style="font-size:1.2em">'
    
    headr = div_header is not None
    if use_class_as_header and not headr:
        div_header = div_class.capitalize()
        headr = True
        
    if headr:
        div_header = div_header.capitalize()
        div += f'<b>{div_header}</b>'
        
    if div_text is not None:
        div_text =  div_text[0].upper() + div_text[1:]
        if headr:
            div += f'<br>&emsp;&emsp;{div_text}'
        else:
            div += f'&ensp;{div_text}'
    
    div += '</p></div>'

    return ipy_set_next_input(div, replace_cell)

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

<div class="alert alert-info"><p style="font-size:1.2em"></p></div>